In [ ]:
from base import generator
from base import generator_haar
from base import epsilon_rho
from base import metrics
from base import optimize_algorithm as optimize
import numpy as np

In [ ]:
def normalize_density_matrix(rho):
    # Calculate the trace of the density matrix
    trace = np.trace(rho)
    
    # Normalize the density matrix
    if trace != 0:
        normalized_rho = rho / trace
    else:
        raise ValueError("The density matrix has a trace of zero and cannot be normalized.")
    
    return normalized_rho

In [ ]:
def quantum_process_tomography_dephasing (num_qubits, gamma, alpha, num_loop, rho, unitary):
    rho2 = epsilon_rho.calculate_dephasing(rho, num_qubits, gamma)
    # Optimizing
    out_unitary, costdict = optimize.optimize_adam_unitary_dagger(rho=rho, rho2=rho2, unitary=unitary, alpha=alpha, num_loop=num_loop)
    
    _cost = costdict[-1]
    return rho2, out_unitary, _cost, costdict

def quantum_process_tomography_unitary (num_qubits, num_loop, alpha, rho, kraus_operators, unitary):
    
    # Optimizing
    out_kraus_operators, costdict = optimize.optimize_adam_kraus(rho=rho, kraus_operators=kraus_operators, unitary=unitary, num_qubits=num_qubits, alpha=alpha, num_loop=num_loop)
    rho2 = epsilon_rho.calculate_from_kraus_operators(rho, out_kraus_operators)
    rho3 = epsilon_rho.calculate_from_unitary_dagger(rho2, unitary)
    _cost = costdict[-1]
    return rho2, out_kraus_operators, _cost, costdict, rho3


In [ ]:
import os
np.set_printoptions(threshold=np.inf)
import re
def write_result (folder, num_qubits, g, alpha, num_loop, rho, unitary):
        folder_path = str(num_qubits)+"_qubits_"+"{:.3f}".format(g)
        folder_path = os.path.join(folder, folder_path)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        rho2, out_unitary, _cost, costdict = quantum_process_tomography_dephasing(num_qubits, g, alpha, num_loop, rho, unitary)
        
        rho2_file_path = os.path.join(folder_path, 'rho2.txt')
        unitary_file_path = os.path.join(folder_path, 'unitary.txt')
        cost_file_path = os.path.join(folder_path, 'cost.txt')
        fidelity22_file_path = os.path.join(folder_path, 'fidelity_rho2_rho2.txt')
        fidelity03_file_path = os.path.join(folder_path, 'fidelity_rho_rho3.txt')
        costdict_file_path = os.path.join(folder_path, 'costdict.txt')

        print(g, num_qubits, _cost)
        
        rho3 = epsilon_rho.calculate_from_unitary_dagger(rho2, out_unitary)
        out_rho2 = epsilon_rho.calculate_from_unitary(rho, out_unitary)

        out_rho2 = normalize_density_matrix(out_rho2)
        #rho3 = normalize_density_matrix(rho3)
        
        with open(rho2_file_path, 'w') as file:
            file.write(str(rho2))
        with open(unitary_file_path, 'w') as file:
            file.write(str(out_unitary.numpy()))
        with open(cost_file_path, 'w') as file:
            file.write(str(_cost.real))
        with open(fidelity03_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho, rho3).numpy()))
        with open(fidelity22_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho2, out_rho2).numpy()))
        with open(costdict_file_path, 'w') as file:
            file.write(str(costdict))


def write_result_unitary (folder, num_qubits, alpha):
        folder_path = str(num_qubits)+"_qubits"
        folder_path = os.path.join(folder, folder_path)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        
        rho = generator_haar.generate_rho_haar(num_qubits)
        unitary = generator_haar.haar(2**num_qubits)   
        kraus_operators = generator_haar.generate_kraus_operators(unitary)
    
        unitary = generator_haar.random_unitary(2**num_qubits)      

        rho2, out_kraus_operators, _cost, costdict, rho3 = quantum_process_tomography_unitary(num_qubits, 1000, alpha, rho, kraus_operators, unitary)
        
        rho_file_path = os.path.join(folder_path, "rho.txt")
        rho2_file_path = os.path.join(folder_path, 'rho2.txt')
        out_rho2_file_path = os.path.join(folder_path, 'out_rho2.txt')
        rho3_file_path = os.path.join(folder_path, 'rho3.txt')
        kraus_file_path = os.path.join(folder_path, 'kraus_operators.txt')
        cost_file_path = os.path.join(folder_path, 'cost.txt')
        fidelity22_file_path = os.path.join(folder_path, 'fidelity_rho2_rho2.txt')
        fidelity03_file_path = os.path.join(folder_path, 'fidelity_rho_rho3.txt')
        costdict_file_path = os.path.join(folder_path, 'costdict.txt')
        unitary_file_path = os.path.join(folder_path, "unitary.txt")

        print(num_qubits, _cost, costdict)
        out_rho2 = epsilon_rho.calculate_from_kraus_operators(rho, out_kraus_operators)
        out_rho2 = normalize_density_matrix(out_rho2)
        #rho3 = normalize_density_matrix(rho3)
        
        with open(fidelity03_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho, rho3).numpy()))
        with open(fidelity22_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho2, out_rho2).numpy()))        
        with open(unitary_file_path, 'w') as file:
            file.write(str(unitary))
        with open(rho_file_path, 'w') as file:
            file.write(str(rho.data))
        with open(rho2_file_path, 'w') as file:
            file.write(str(rho2))
        with open(out_rho2_file_path, 'w') as file:
            file.write(str(out_rho2.numpy()))
        with open(rho3_file_path, 'w') as file:
            file.write(str(rho3.numpy()))
        with open(kraus_file_path, 'w') as file:
            file.write(str(out_kraus_operators.numpy()))
        with open(cost_file_path, 'w') as file:
            file.write(str(_cost))
        
        with open(costdict_file_path, 'w') as file:
            file.write(str(costdict))





In [ ]:
experiment_folder = ''
g_shift = 0.05

for num_qubits in range(1, 5):
    if (experiment_folder == ''):
        break
    else:
        if not os.path.exists(experiment_folder):
            os.makedirs(experiment_folder)

    write_result_unitary(experiment_folder, num_qubits, 0.03)
    
    

In [ ]:
experiment_folder = ''
g_shift = 0.05

for num_qubits in range(1, 2):
    if (experiment_folder == ''):
        break
    else:
        if not os.path.exists(experiment_folder):
            os.makedirs(experiment_folder)
    rho = generator_haar.generate_rho_haar(num_qubits)
    unitary_matrix = generator_haar.random_unitary(2**num_qubits)

    rho_file_path = os.path.join(experiment_folder, str(num_qubits)+"_qubits_rho.txt")
    with open(rho_file_path, 'w') as file:
        file.write(str(rho.data))
    g = 1
    while g >= 0:
        write_result(experiment_folder, num_qubits, g, 0.01, 1000, rho, unitary_matrix)
        g = round(g - g_shift, 2)
    
    

In [ ]:
import os
def write_result_2(folder, folder_path, num_qubits, g, alpha, num_loop, rho, unitary): 
    folder_path = os.path.join(folder, folder_path)
    os.makedirs(folder_path, exist_ok=True)  # Create directory if it doesn't exist

    rho2, out_unitary, _cost, costdict = quantum_process_tomography_dephasing(num_qubits, g, alpha, num_loop, rho, unitary)
    
    # File paths
    rho2_file_path = os.path.join(folder_path, 'rho2.txt')
    unitary_file_path = os.path.join(folder_path, 'unitary.txt')
    trace_rho2_file_path = os.path.join(folder_path, 'tr_rho2.txt')
    trace_out_rho2_file_path = os.path.join(folder_path, 'tr_out_rho2.txt')
    fidelity_file_path = os.path.join(folder_path, 'fidelity.txt')

    out_rho2 = epsilon_rho.calculate_from_unitary(rho, out_unitary)
    out_rho2 = normalize_density_matrix(out_rho2)
    
    # Define Pauli-X matrix
    pauli_X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
    # Calculate tr(X1, rho)
    trace_rho2 = metrics.trace_Pauli(rho2, 0, pauli_X)
    trace_out_rho2 = metrics.trace_Pauli(out_rho2, 0, pauli_X)
    
    print(g, num_qubits, _cost)
    print(costdict)
    # Save results
    with open(rho2_file_path, 'w') as file:
        file.write(str(rho2))
    with open(fidelity_file_path, 'w') as file:
        file.write(str(metrics.compilation_trace_fidelity
                       (rho, epsilon_rho.calculate_from_unitary_dagger(rho2, out_unitary)).numpy().real))
    with open(unitary_file_path, 'w') as file:
        file.write(str(out_unitary.numpy()))
    with open(trace_rho2_file_path, 'w') as file:
        file.write(str(trace_rho2.real))
    with open(trace_out_rho2_file_path, 'w') as file:
        file.write(str(trace_out_rho2.real))

In [ ]:
from qiskit.quantum_info import DensityMatrix
experiment_folder = ''
# Đổi hàm cost thành trace fidelity
for num_qubits in range(1, 2):
    if (experiment_folder == ''):
        break
    else:
        if not os.path.exists(experiment_folder):
            os.makedirs(experiment_folder)
    rho = DensityMatrix([[0.99817499+0.j, 0.04268112+0.j], [0.04268112+0.j, 0.00182501+0.j]])
    unitary_matrix = [[-0.07737892+0.j,  0.99700176+0.j], [ 0.99700176+0.j,  0.07737892+0.j]]

    rho_file_path = os.path.join(experiment_folder, str(num_qubits)+"_qubits_rho.txt")
    with open(rho_file_path, 'w') as file:
        file.write(str(rho.data))
    unitary_file_path = os.path.join(experiment_folder, str(num_qubits)+"_UNITARY.txt")
    with open(unitary_file_path, 'w') as file:
        file.write(str(unitary_matrix))
    g = 0
    beta = 0.1
    # Create 50 t from 0 to 48
    t_s = np.linspace(74, 100, 53)
    for t in t_s:
        t=round(t, 1)
        print(g, num_qubits, t)
        folder_path_a = str(num_qubits)+"_qubits_"+"{:07.2f}".format(t)+'_a'
        folder_path_b = str(num_qubits)+"_qubits_"+"{:07.2f}".format(t)+'_b'
        write_result_2(experiment_folder, folder_path_a, 
                        num_qubits, 1 - np.exp(-2 * beta * t), 0.01, 700, rho, unitary_matrix)
        write_result_2(experiment_folder, folder_path_b,
                        num_qubits, 1 - np.exp(-2 * beta * t**2), 0.01, 700, rho, unitary_matrix)
    
    
